In [ ]:
import mitsuba as mi
import torch
from matplotlib import pyplot as plt
import numpy as np
from prettytable import PrettyTable
from pathlib import Path
from tqdm import tqdm
mi.set_variant('cuda_ad_rgb')
import nerad.integrator
from nerad.utils.sensor_utils import create_transforms
from nerad.utils.json_utils import write_json
from nerad.utils.image_utils import save_image

In [ ]:
def plot_list(images, titles: list[str], figure_title: str):
    fig, axs = plt.subplots(1, len(images), figsize=(18, 3))
    for i in range(len(images)):
        axs[i].imshow(mi.util.convert_to_bitmap(images[i]))
        axs[i].axis('off')
        if titles is not None:
            axs[i].set_title(titles[i])
    if figure_title is not None:
        fig.suptitle(figure_title)
    
    plt.show()
    plt.close(fig)

In [ ]:
max_depth = 1024
rr_depth = 1
rr_prob = 1.0
spp = 1

integrator = mi.load_dict({
    'type': 'mypath',
    'config': {
        'type': 'dict',
        'hide_emitters': True,
        'return_depth': True,
        'max_depth': max_depth,
        'rr_depth': rr_depth,
        'rr_prob': rr_prob,
    }})

In [ ]:
albedos = [0.1, 0.3, 0.5, 0.7, 0.9, 0.95, 0.97, 0.99]
path_lengths = {}
images = {}
for alb in albedos:
    scene = mi.load_file('/data/nerad/mitsuba3_scenes/cube_scene/scene.xml', albedo=alb)
    img = mi.render(scene, integrator=integrator, spp=spp)

    path_lengths[alb] = img[:, :, -1].torch().cpu().view(-1).numpy()
    images[alb] = img[:, :, :3]

plot_list([images[v] for v in albedos], [str(v) for v in albedos], 'vary albedo')

In [ ]:
def render_gt():
    out_root = Path("/data/nerad/datasets/cube_scene")
    scene_root = Path("/data/nerad/mitsuba3_scenes/cube_scene")
    sensors = mi.load_file(str(scene_root / "cameras.xml")).sensors()
    integrator = mi.load_dict({
        'type': 'mypath',
        'config': {
            'type': 'dict',
            'hide_emitters': True,
            'return_depth': False,
            'max_depth': 1024,
            'rr_depth': 1,
            'rr_prob': 1.0,
        }})

    transforms = create_transforms(str(scene_root / "scene.xml"), len(sensors))
    for alb in albedos:
        alb_folder = out_root / f"albedo_{alb}"
        alb_folder.mkdir(parents=True, exist_ok=True)
        write_json(alb_folder / "transforms.json", transforms)

        print(f"Render albedo = {alb}")
        scene = mi.load_file(str(scene_root / "scene.xml"), albedo=alb)
        for i, sensor in enumerate(tqdm(sensors)):
            img = mi.render(scene, integrator=integrator, spp=512, sensor=sensor)
            save_image(alb_folder, f"{i:03d}", ["png", "exr"], img)

## Analyze Path Lengths

In [ ]:
percentiles = [1, 25, 50, 75, 99, 99.9]
path_length_data_by_albedo = {}  # albedo -> dict[str, value]
path_length_data_keys = ["Minimum", "Maximum", "Average", *(f"{p}%" for p in percentiles)]

for alb in albedos:
    data = path_lengths[alb]
    perc = np.percentile(data, percentiles)
    path_length_data_by_albedo[alb] = {
        "Minimum": np.min(data),
        "Maximum": np.max(data),
        "Average": np.mean(data),
        **{
            f"{p}%": perc[j] for j, p in enumerate(percentiles)
        }
    }

tab = PrettyTable(["Albedo", *(str(alb) for alb in albedos)])
for key in path_length_data_keys:
    tab.add_row([
        key, *(f"{path_length_data_by_albedo[v][key]:.3f}" for v in albedos)
    ])
tab.align = "r"
print(f"spp = {spp}")
print(tab)

In [ ]:
for key in ["1%", "50%", "99%", "99.9%"]:
    plt.plot(np.array(albedos), np.array([path_length_data_by_albedo[v][key] for v in albedos]), label = key)
plt.legend()
plt.xlabel('albedos')
plt.ylabel('path length')
plt.show()